1. **Task: Bank Transactions**
You are tasked with implementing a banking system that allows customers to transfer funds between accounts.
Design a SQL transaction to ensure that the amount is deducted from the sender's account and added to the receiver's account in an atomic manner.
If any part of the transaction fails, rollback the changes to maintain data integrity.

In [12]:
import MySQLdb
import random
import string

Tables and data

In [65]:
def generate_sample_data():
    account_numbers = list(range(1001, 1011))

    balances = [random.uniform(500, 5000) for _ in range(len(account_numbers))]

    transactions = []
    for _ in range(10):
        sender_account = random.choice(account_numbers)
        receiver_account = random.choice(account_numbers)
        while receiver_account == sender_account:
            receiver_account = random.choice(account_numbers)
        amount = random.uniform(10, 100)
        transactions.append((sender_account, receiver_account, amount))

    return account_numbers, balances, transactions

def create_tables():
    conn = MySQLdb.connect(host='localhost',
                           user='root',
                           password='1234',
                           database='advanced_sql')
    cursor = conn.cursor()

    try:
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS accounts_task1 (
                account_number INT PRIMARY KEY,
                balance DECIMAL
            )
        ''')

        cursor.execute('''
            CREATE TABLE IF NOT EXISTS transactions_task1 (
                id INT PRIMARY KEY AUTO_INCREMENT,
                sender_account INT,
                receiver_account INT,
                amount DECIMAL,
                FOREIGN KEY (sender_account) REFERENCES accounts_task1(account_number),
                FOREIGN KEY (receiver_account) REFERENCES accounts_task1(account_number)
            )
        ''')

        print('Tables created successfully!')

    except Exception as e:
        print(f'Error creating tables: {e}')

    finally:
        conn.close()

def insert_sample_data(account_numbers, balances, transactions):
    conn = MySQLdb.connect(host='localhost',
                           user='root',
                           password='1234',
                           database='advanced_sql')
    cursor = conn.cursor()

    try:
        for account_number, balance in zip(account_numbers, balances):
            cursor.execute('INSERT INTO accounts_task1 (account_number, balance) VALUES (%s, %s)', (account_number, balance))

        for sender_account, receiver_account, amount in transactions:
            cursor.execute('INSERT INTO transactions_task1 (sender_account, receiver_account, amount) VALUES (%s, %s, %s)',
                           (sender_account, receiver_account, amount))

        conn.commit()
        print('Sample data inserted successfully!')

    except Exception as e:
        conn.rollback()
        print(f'Error inserting sample data: {e}')

    finally:
        conn.close()


account_numbers, balances, transactions = generate_sample_data()
create_tables()
insert_sample_data(account_numbers, balances, transactions)


Tables created successfully!
Sample data inserted successfully!


In [68]:
def transfer_funds(sender_account, receiver_account, amount):
    conn = MySQLdb.connect(host='localhost',
                           user='root',
                           password='1234',
                           database='advanced_sql')
    cursor = conn.cursor()

    try:
        conn.begin()

        # Check if the sender's account has enough balance
        cursor.execute('SELECT balance FROM accounts_task1 WHERE account_number = %s FOR UPDATE', (sender_account,))
        sender_balance = cursor.fetchone()[0]
        if sender_balance < amount:
            raise ValueError("Insufficient funds in the sender's account.")

        # Deduct the amount from the sender's account
        cursor.execute('UPDATE accounts_task1 SET balance = balance - %s WHERE account_number = %s', (amount, sender_account))

        # Add the amount to the receiver's account
        cursor.execute('UPDATE accounts_task1 SET balance = balance + %s WHERE account_number = %s', (amount, receiver_account))

        conn.commit()

        print(f'Funds transferred successfully. {amount} units transferred from account {sender_account} to account {receiver_account}.')

    except Exception as e:
        conn.rollback()
        print(f'Transfer failed. Error: {e}')

    finally:
        cursor.close()
        conn.close()


sender_account = 1001
receiver_account = 1002
amount = 100.0
transfer_funds(sender_account, receiver_account, amount)

Funds transferred successfully. 100.0 units transferred from account 1001 to account 1002.


2. **Task: Order Processing**
You are developing an online shopping platform.
Create a SQL transaction that updates the inventory of products when a customer places an order.
Deduct the ordered quantity from the product stock and update the order status.
If any part of the transaction fails (e.g., insufficient stock), roll back the changes to ensure consistent data.

Tables and data

In [20]:
def create_tables():
    conn = MySQLdb.connect(host='localhost',
                           user='root',
                           password='1234',
                           database='advanced_sql')
    cursor = conn.cursor()

    try:
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS products_task2 (
                product_id INT PRIMARY KEY AUTO_INCREMENT,
                product_name VARCHAR(255) NOT NULL,
                stock INT NOT NULL
            )
        ''')

        cursor.execute('''
            CREATE TABLE IF NOT EXISTS orders_task2 (
                order_id INT PRIMARY KEY AUTO_INCREMENT,
                customer_id INT NOT NULL,
                order_status VARCHAR(50) NOT NULL
            )
        ''')

        cursor.execute('''
            CREATE TABLE IF NOT EXISTS order_items_task2 (
                item_id INT PRIMARY KEY AUTO_INCREMENT,
                order_id INT NOT NULL,
                product_id INT NOT NULL,
                quantity INT NOT NULL,
                FOREIGN KEY (order_id) REFERENCES orders_task2(order_id),
                FOREIGN KEY (product_id) REFERENCES products_task2(product_id)
            )
        ''')

        print('Tables created successfully!')

        conn.commit()

    except Exception as e:
        conn.rollback()
        print(f'Error creating tables: {e}')

    finally:
        conn.close()

def generate_random_product_data(num_products):
    products = []
    for _ in range(num_products):
        product_name = ''.join(random.choices(string.ascii_letters, k=random.randint(5, 15)))
        stock = random.randint(50, 200)
        products.append((product_name, stock))
    return products

def generate_random_order_data(num_orders, product_ids):
    orders = []
    for _ in range(num_orders):
        customer_id = random.randint(1000, 9999)
        order_status = random.choice(['Pending', 'Shipped', 'Delivered'])
        product_id = random.choice(product_ids)
        quantity = random.randint(1, 10)
        orders.append((customer_id, order_status, product_id, quantity))
    return orders

def insert_random_product_data():
    conn = MySQLdb.connect(host='localhost',
                           user='root',
                           password='1234',
                           database='advanced_sql')
    cursor = conn.cursor()

    try:
        random_products = generate_random_product_data(10)

        cursor.executemany('INSERT INTO products_task2 (product_name, stock) VALUES (%s, %s)', random_products)

        conn.commit()
        print('Random product data inserted successfully!')

    except Exception as e:
        conn.rollback()
        print(f'Error inserting random product data: {e}')

    finally:
        conn.close()

def insert_random_order_data(product_ids):
    conn = MySQLdb.connect(host='localhost',
                           user='root',
                           password='1234',
                           database='advanced_sql')
    cursor = conn.cursor()

    try:
        random_orders = generate_random_order_data(10, product_ids)

        for customer_id, order_status, product_id, quantity in random_orders:
            cursor.execute('INSERT INTO orders_task2 (customer_id, order_status) VALUES (%s, %s)', (customer_id, order_status))
            order_id = cursor.lastrowid
            cursor.execute('INSERT INTO order_items_task2 (order_id, product_id, quantity) VALUES (%s, %s, %s)',
                           (order_id, product_id, quantity))

        conn.commit()
        print('Random order data inserted successfully!')

    except Exception as e:
        conn.rollback()
        print(f'Error inserting random order data: {e}')

    finally:
        conn.close()

create_tables()
insert_random_product_data()
insert_random_order_data(list(range(1, 11)))

Tables created successfully!
Random product data inserted successfully!
Random order data inserted successfully!


In [25]:
def place_order(customer_id, product_id, quantity):
    conn = MySQLdb.connect(host='localhost',
                           user='root',
                           password='1234',
                           database='advanced_sql')
    cursor = conn.cursor()

    try:
        conn.begin()

        # Check if there is enough stock for the product
        cursor.execute('SELECT stock FROM products_task2 WHERE product_id = %s FOR UPDATE', (product_id,))
        current_stock = cursor.fetchone()[0]
        if current_stock < quantity:
            raise ValueError("Insufficient stock for the product.")

        # Deduct the ordered quantity from the product stock
        updated_stock = current_stock - quantity
        cursor.execute('UPDATE products_task2 SET stock = %s WHERE product_id = %s', (updated_stock, product_id))

        # Insert the order into the orders table
        cursor.execute('INSERT INTO orders_task2 (customer_id, order_status) VALUES (%s, %s)', (customer_id, 'Pending'))
        order_id = cursor.lastrowid

        # Insert the order items into the order_items table
        cursor.execute('INSERT INTO order_items_task2 (order_id, product_id, quantity) VALUES (%s, %s, %s)',
                       (order_id, product_id, quantity))

        conn.commit()

        print(f'Order placed successfully. Order ID: {order_id}')

    except Exception as e:
        conn.rollback()
        print(f'Order placement failed. Error: {e}')

    finally:
        cursor.close()
        conn.close()

customer_id = 12345
product_id = 1
quantity = 1000
place_order(customer_id, product_id, quantity)

Order placement failed. Error: Insufficient stock for the product.


3. **Task: Employee Salary Adjustment**
Your HR department needs to apply a salary adjustment to all employees based on their performance.
Design a SQL transaction that updates the salary of each employee with the corresponding adjustment and logs the changes. If there is any issue while updating salaries, roll back the changes to maintain the previous salary data.

Tables and data

In [26]:
def generate_random_employee_data(num_employees):
    employees = []
    for _ in range(num_employees):
        first_name = ''.join(random.choices(string.ascii_letters, k=random.randint(5, 10)))
        last_name = ''.join(random.choices(string.ascii_letters, k=random.randint(5, 10)))
        salary = random.randint(30000, 80000)
        employees.append((first_name, last_name, salary))
    return employees

def create_tables():
    conn = MySQLdb.connect(host='localhost',
                           user='root',
                           password='1234',
                           database='advanced_sql')
    cursor = conn.cursor()

    try:
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS employees_task3 (
                employee_id INT PRIMARY KEY AUTO_INCREMENT,
                first_name VARCHAR(50) NOT NULL,
                last_name VARCHAR(50) NOT NULL,
                salary INT NOT NULL
            )
        ''')

        cursor.execute('''
            CREATE TABLE IF NOT EXISTS salary_adjustments_task3 (
                adjustment_id INT PRIMARY KEY AUTO_INCREMENT,
                percentage_adjustment FLOAT NOT NULL
            )
        ''')

        cursor.execute('''
            CREATE TABLE IF NOT EXISTS salary_logs_task3 (
                log_id INT PRIMARY KEY AUTO_INCREMENT,
                employee_id INT NOT NULL,
                old_salary INT NOT NULL,
                new_salary INT NOT NULL,
                adjustment_id INT NOT NULL,
                FOREIGN KEY (employee_id) REFERENCES employees_task3(employee_id),
                FOREIGN KEY (adjustment_id) REFERENCES salary_adjustments_task3(adjustment_id)
            )
        ''')

        print('Tables created successfully!')

        conn.commit()

    except Exception as e:
        conn.rollback()
        print(f'Error creating tables: {e}')

    finally:
        conn.close()

def insert_random_employee_data():
    conn = MySQLdb.connect(host='localhost',
                           user='root',
                           password='1234',
                           database='advanced_sql')
    cursor = conn.cursor()

    try:
        random_employees = generate_random_employee_data(10)

        cursor.executemany('INSERT INTO employees_task3 (first_name, last_name, salary) VALUES (%s, %s, %s)', random_employees)

        conn.commit()
        print('Random employee data inserted successfully!')

    except Exception as e:
        conn.rollback()
        print(f'Error inserting random employee data: {e}')

    finally:
        conn.close()

create_tables()
insert_random_employee_data()

Tables created successfully!
Random employee data inserted successfully!


In [27]:
def apply_salary_adjustment(percentage_adjustment):
    conn = MySQLdb.connect(host='localhost',
                           user='root',
                           password='1234',
                           database='advanced_sql')
    cursor = conn.cursor()

    try:
        conn.begin()

        # Get the current employee salaries
        cursor.execute('SELECT employee_id, salary FROM employees_task3 FOR UPDATE')
        employee_salaries = cursor.fetchall()

        # Apply salary adjustment to each employee and log the changes
        for employee_id, current_salary in employee_salaries:
            new_salary = int(current_salary * (1 + percentage_adjustment))
            cursor.execute('INSERT INTO salary_adjustments_task3 (percentage_adjustment) VALUES (%s)', (percentage_adjustment,))
            adjustment_id = cursor.lastrowid
            cursor.execute('INSERT INTO salary_logs_task3 (employee_id, old_salary, new_salary, adjustment_id) VALUES (%s, %s, %s, %s)',
                           (employee_id, current_salary, new_salary, adjustment_id))
            cursor.execute('UPDATE employees SET salary = %s WHERE employee_id = %s', (new_salary, employee_id))

        conn.commit()
        print('Salary adjustment applied successfully!')

    except Exception as e:
        conn.rollback()
        print(f'Error applying salary adjustment: {e}')

    finally:
        conn.close()

apply_salary_adjustment(0.05)

Salary adjustment applied successfully!


4. **Task: Flight Reservation**
You are building a flight reservation system.
Create a SQL transaction that reserves a seat for a passenger on a flight.
Deduct the seat availability, update the reservation record, and log the booking details.
If any part of the transaction fails (e.g., seat already taken),
roll back the changes and notify the user of the failure.


In [28]:
def generate_random_flight_data(num_flights, num_seats_per_flight):
    flights = []
    for flight_id in range(1, num_flights + 1):
        flight_name = ''.join(random.choices(string.ascii_uppercase, k=random.randint(5, 10)))
        num_available_seats = num_seats_per_flight
        flights.append((flight_id, flight_name, num_available_seats))
    return flights

def generate_random_passenger_data(num_passengers):
    passengers = []
    for _ in range(num_passengers):
        first_name = ''.join(random.choices(string.ascii_letters, k=random.randint(5, 10)))
        last_name = ''.join(random.choices(string.ascii_letters, k=random.randint(5, 10)))
        passengers.append((first_name, last_name))
    return passengers

def create_tables():
    conn = MySQLdb.connect(host='localhost',
                           user='root',
                           password='1234',
                           database='advanced_sql')
    cursor = conn.cursor()

    try:
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS flights_task4 (
                flight_id INT PRIMARY KEY,
                flight_name VARCHAR(10) NOT NULL,
                num_available_seats INT NOT NULL
            )
        ''')

        cursor.execute('''
            CREATE TABLE IF NOT EXISTS passengers_task4 (
                passenger_id INT PRIMARY KEY AUTO_INCREMENT,
                first_name VARCHAR(50) NOT NULL,
                last_name VARCHAR(50) NOT NULL
            )
        ''')

        cursor.execute('''
            CREATE TABLE IF NOT EXISTS reservations_task4 (
                reservation_id INT PRIMARY KEY AUTO_INCREMENT,
                flight_id INT NOT NULL,
                passenger_id INT NOT NULL,
                reservation_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                FOREIGN KEY (flight_id) REFERENCES flights_task4(flight_id),
                FOREIGN KEY (passenger_id) REFERENCES passengers_task4(passenger_id)
            )
        ''')

        print('Tables created successfully!')

        conn.commit()

    except Exception as e:
        conn.rollback()
        print(f'Error creating tables: {e}')

    finally:
        conn.close()

def insert_random_flight_data(num_flights, num_seats_per_flight):
    conn = MySQLdb.connect(host='localhost',
                           user='root',
                           password='1234',
                           database='advanced_sql')
    cursor = conn.cursor()

    try:
        random_flights = generate_random_flight_data(num_flights, num_seats_per_flight)

        cursor.executemany('INSERT INTO flights_task4 (flight_id, flight_name, num_available_seats) VALUES (%s, %s, %s)', random_flights)

        conn.commit()
        print('Random flight data inserted successfully!')

    except Exception as e:
        conn.rollback()
        print(f'Error inserting random flight data: {e}')

    finally:
        conn.close()

def insert_random_passenger_data(num_passengers):
    conn = MySQLdb.connect(host='localhost',
                           user='root',
                           password='1234',
                           database='advanced_sql')
    cursor = conn.cursor()

    try:
        random_passengers = generate_random_passenger_data(num_passengers)

        cursor.executemany('INSERT INTO passengers_task4 (first_name, last_name) VALUES (%s, %s)', random_passengers)

        conn.commit()
        print('Random passenger data inserted successfully!')

    except Exception as e:
        conn.rollback()
        print(f'Error inserting random passenger data: {e}')

    finally:
        conn.close()

create_tables()
insert_random_flight_data(5, 500)  # 5 flights with 500 available seats each
insert_random_passenger_data(50)

Tables created successfully!
Random flight data inserted successfully!
Random passenger data inserted successfully!


In [34]:
def reserve_seat(flight_id, passenger_id):
    conn = MySQLdb.connect(host='localhost',
                           user='root',
                           password='1234',
                           database='advanced_sql')
    cursor = conn.cursor()

    try:
        conn.begin()

        cursor.execute('SELECT num_available_seats FROM flights_task4 WHERE flight_id = %s FOR UPDATE', (flight_id,))
        flight_data = cursor.fetchone()

        if not flight_data:
            print('Flight does not exist.')
            return

        cursor.execute('SELECT * FROM passengers_task4 WHERE passenger_id = %s', (passenger_id,))
        passenger_data = cursor.fetchone()

        if not passenger_data:
            print('Passenger does not exist.')
            return

        # Check if there are available seats on the flight
        num_available_seats = flight_data[0]
        if num_available_seats <= 0:
            print('Sorry, no available seats on this flight.')
            return

        # Reserve the seat for the passenger
        cursor.execute('UPDATE flights_task4 SET num_available_seats = num_available_seats - 1 WHERE flight_id = %s', (flight_id,))
        cursor.execute('INSERT INTO reservations_task4 (flight_id, passenger_id) VALUES (%s, %s)', (flight_id, passenger_id))

        conn.commit()
        print('Seat reserved successfully!')

    except Exception as e:
        conn.rollback()
        print(f'Error reserving seat: {e}')

    finally:
        conn.close()

reserve_seat(1, 2)

Seat reserved successfully!


5. **Task: Product Returns**
You are developing an e-commerce platform.
Implement a SQL transaction to process product returns initiated by customers.
Update the product's return status, refund the customer's payment, and log the return details.
If any part of the transaction fails (e.g., invalid return request),
roll back the changes and handle the error gracefully.


Tables and data

In [50]:
def generate_random_product_data(num_products):
    products = []
    for _ in range(num_products):
        product_name = ''.join(random.choices(string.ascii_letters, k=random.randint(5, 15)))
        price = round(random.uniform(10, 1000), 2)
        products.append((product_name, price))
    return products

def generate_random_customer_data(num_customers):
    customers = []
    for _ in range(num_customers):
        first_name = ''.join(random.choices(string.ascii_letters, k=random.randint(5, 10)))
        last_name = ''.join(random.choices(string.ascii_letters, k=random.randint(5, 10)))
        customers.append((first_name, last_name))
    return customers

def create_tables():
    conn = MySQLdb.connect(host='localhost',
                           user='root',
                           password='1234',
                           database='advanced_sql')
    cursor = conn.cursor()

    try:
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS products_task5 (
                product_id INT PRIMARY KEY AUTO_INCREMENT,
                product_name VARCHAR(255) NOT NULL,
                price DECIMAL(10, 2) NOT NULL,
                return_status VARCHAR(50),
                INDEX (product_name)
            )
        ''')

        cursor.execute('''
            CREATE TABLE IF NOT EXISTS customers_task5 (
                customer_id INT PRIMARY KEY AUTO_INCREMENT,
                first_name VARCHAR(50) NOT NULL,
                last_name VARCHAR(50) NOT NULL
            )
        ''')

        cursor.execute('''
            CREATE TABLE IF NOT EXISTS returns_task5 (
                return_id INT PRIMARY KEY AUTO_INCREMENT,
                customer_id INT NOT NULL,
                product_id INT NOT NULL,
                return_status VARCHAR(50) DEFAULT 'Not Returned',
                refund_amount DECIMAL(10, 2) NOT NULL,
                return_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                FOREIGN KEY (customer_id) REFERENCES customers_task5(customer_id),
                FOREIGN KEY (product_id) REFERENCES products_task5(product_id)
            )
        ''')

        print('Tables created successfully!')

        conn.commit()

    except Exception as e:
        conn.rollback()
        print(f'Error creating tables: {e}')

    finally:
        conn.close()

def insert_random_product_data(num_products):
    conn = MySQLdb.connect(host='localhost',
                           user='root',
                           password='1234',
                           database='advanced_sql')
    cursor = conn.cursor()

    try:
        random_products = generate_random_product_data(num_products)

        cursor.executemany('INSERT INTO products_task5 (product_name, price) VALUES (%s, %s)', random_products)

        conn.commit()
        print('Random product data inserted successfully!')

    except Exception as e:
        conn.rollback()
        print(f'Error inserting random product data: {e}')

    finally:
        conn.close()

def insert_random_customer_data(num_customers):
    conn = MySQLdb.connect(host='localhost',
                           user='root',
                           password='1234',
                           database='advanced_sql')
    cursor = conn.cursor()

    try:
        random_customers = generate_random_customer_data(num_customers)

        cursor.executemany('INSERT INTO customers_task5 (first_name, last_name) VALUES (%s, %s)', random_customers)

        conn.commit()
        print('Random customer data inserted successfully!')

    except Exception as e:
        conn.rollback()
        print(f'Error inserting random customer data: {e}')

    finally:
        conn.close()

create_tables()
insert_random_product_data(5)
insert_random_customer_data(50)

Tables created successfully!
Random product data inserted successfully!
Random customer data inserted successfully!


In [51]:
def process_product_return(customer_id, product_id, refund_amount):
    conn = MySQLdb.connect(host='localhost',
                           user='root',
                           password='1234',
                           database='advanced_sql')
    cursor = conn.cursor()

    try:
        conn.begin()

        cursor.execute('SELECT * FROM customers_task5 WHERE customer_id = %s', (customer_id,))
        customer_data = cursor.fetchone()

        if not customer_data:
            print('Customer does not exist.')
            return

        cursor.execute('SELECT * FROM products_task5 WHERE product_id = %s', (product_id,))
        product_data = cursor.fetchone()

        if not product_data:
            print('Product does not exist.')
            return

        # Update the product's return status and refund the customer's payment
        cursor.execute('UPDATE products_task5 SET return_status = %s WHERE product_id = %s', ('Returned', product_id))
        cursor.execute('INSERT INTO returns_task5 (customer_id, product_id, return_status, refund_amount) VALUES (%s, %s, %s, %s)',
                       (customer_id, product_id, 'Returned', refund_amount))

        conn.commit()
        print('Product return processed successfully!')

    except Exception as e:
        conn.rollback()
        print(f'Error processing product return: {e}')

    finally:
        conn.close()

customer_id = 1
product_id = 1
refund_amount = 50.00
process_product_return(customer_id, product_id, refund_amount)

Product return processed successfully!


6. **Task: Student Enrollment**
Design a SQL transaction to handle the enrollment of new students in a university.
Update the student enrollment records, assign courses, and update the tuition fee payment status. 
If any part of the transaction fails (e.g., course limit reached), roll back the changes to ensure no invalid enrollments are recorded.


Tables and data

In [53]:
def generate_random_student_data(num_students):
    students = []
    for _ in range(num_students):
        first_name = ''.join(random.choices(string.ascii_letters, k=random.randint(5, 10)))
        last_name = ''.join(random.choices(string.ascii_letters, k=random.randint(5, 10)))
        tuition_fee_paid = random.choice([True, False])
        students.append((first_name, last_name, tuition_fee_paid))
    return students

def generate_random_course_data(num_courses):
    courses = []
    for _ in range(num_courses):
        course_name = ''.join(random.choices(string.ascii_uppercase, k=random.randint(5, 10)))
        capacity = random.randint(5, 30)
        courses.append((course_name, capacity))
    return courses

def create_tables():
    conn = MySQLdb.connect(host='localhost',
                           user='root',
                           password='1234',
                           database='advanced_sql')
    cursor = conn.cursor()

    try:
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS students_task6 (
                student_id INT PRIMARY KEY AUTO_INCREMENT,
                first_name VARCHAR(50) NOT NULL,
                last_name VARCHAR(50) NOT NULL,
                tuition_fee_paid BOOLEAN NOT NULL
            )
        ''')

        cursor.execute('''
            CREATE TABLE IF NOT EXISTS courses_task6 (
                course_id INT PRIMARY KEY AUTO_INCREMENT,
                course_name VARCHAR(50) NOT NULL,
                capacity INT NOT NULL
            )
        ''')

        cursor.execute('''
            CREATE TABLE IF NOT EXISTS enrollments_task6 (
                enrollment_id INT PRIMARY KEY AUTO_INCREMENT,
                student_id INT NOT NULL,
                course_id INT NOT NULL,
                enrollment_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                FOREIGN KEY (student_id) REFERENCES students_task6(student_id),
                FOREIGN KEY (course_id) REFERENCES courses_task6(course_id)
            )
        ''')

        print('Tables created successfully!')

        conn.commit()

    except Exception as e:
        conn.rollback()
        print(f'Error creating tables: {e}')

    finally:
        conn.close()

def insert_random_student_data(num_students):
    conn = MySQLdb.connect(host='localhost',
                           user='root',
                           password='1234',
                           database='advanced_sql')
    cursor = conn.cursor()

    try:
        random_students = generate_random_student_data(num_students)

        cursor.executemany('INSERT INTO students_task6 (first_name, last_name, tuition_fee_paid) VALUES (%s, %s, %s)', random_students)

        conn.commit()
        print('Random student data inserted successfully!')

    except Exception as e:
        conn.rollback()
        print(f'Error inserting random student data: {e}')

    finally:
        conn.close()

def insert_random_course_data(num_courses):
    conn = MySQLdb.connect(host='localhost',
                           user='root',
                           password='1234',
                           database='advanced_sql')
    cursor = conn.cursor()

    try:
        random_courses = generate_random_course_data(num_courses)

        cursor.executemany('INSERT INTO courses_task6 (course_name, capacity) VALUES (%s, %s)', random_courses)

        conn.commit()
        print('Random course data inserted successfully!')

    except Exception as e:
        conn.rollback()
        print(f'Error inserting random course data: {e}')

    finally:
        conn.close()

create_tables()
insert_random_student_data(20)
insert_random_course_data(5)

Tables created successfully!
Random student data inserted successfully!
Random course data inserted successfully!


In [55]:
def enroll_student(student_id, course_id):
    conn = MySQLdb.connect(host='localhost',
                           user='root',
                           password='1234',
                           database='advanced_sql')
    cursor = conn.cursor()

    try:
        conn.begin()

        # Check if the student and course exist
        cursor.execute('SELECT * FROM students_task6 WHERE student_id = %s', (student_id,))
        student_data = cursor.fetchone()

        if not student_data:
            print('Student does not exist.')
            return

        cursor.execute('SELECT * FROM courses_task6 WHERE course_id = %s', (course_id,))
        course_data = cursor.fetchone()

        if not course_data:
            print('Course does not exist.')
            return

        # Check if the course has available capacity
        capacity = course_data[2]
        cursor.execute('SELECT COUNT(*) FROM enrollments_task6 WHERE course_id = %s', (course_id,))
        num_enrollments = cursor.fetchone()[0]

        if num_enrollments >= capacity:
            print('Course capacity reached. Enrollment not allowed.')
            return

        # Update the tuition_fee_paid status for the student
        cursor.execute('UPDATE students_task6 SET tuition_fee_paid = TRUE WHERE student_id = %s', (student_id,))

        # Enroll the student in the course
        cursor.execute('INSERT INTO enrollments_task6 (student_id, course_id) VALUES (%s, %s)', (student_id, course_id))

        conn.commit()
        print('Student enrolled successfully!')

    except Exception as e:
        conn.rollback()
        print(f'Error enrolling student: {e}')

    finally:
        conn.close()

student_id = 1
course_id = 3

enroll_student(student_id, course_id)

Student enrolled successfully!
